# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [9]:
!pip install pandas

In [13]:
import pandas as pd
df = pd.read_csv('C:/Users/Ignacio/IronHackCodes/gitHStuff/Labs/lab-chains-in-langchain-main/data/Data.csv')

In [15]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [25]:
!pip install langchain

In [27]:
!pip install openai

In [29]:
!pip install langchain openai

In [41]:
!pip install --upgrade langchain openai
!pip install langchain_community

  Attempting uninstall: openai
    Found existing installation: openai 1.59.6
    Uninstalling openai-1.59.6:
      Successfully uninstalled openai-1.59.6
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------------------- ------------------- 1.3/2.5 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 16.0 MB/s eta 0:00:00


In [273]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [275]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7) #0.7 = creative


In [277]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
 "you are given the task to name the price, size and color of the product '{product_name}' "
)

In [279]:
chain = LLMChain(llm=llm, prompt=prompt)

In [281]:
product = {"product_name":"smartphone"} 

In [283]:
chain.run(product)

'Price: $699\nSize: 6.1 inches\nColor: Midnight Black'

## SimpleSequentialChain

In [285]:
from langchain.chains import SimpleSequentialChain

In [287]:
llm = ChatOpenAI(temperature=0.9)

product = {"input": "Price: $699\nSize: 6.1 inches\nColor: Midnight Black"}

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
    template="You are a customer that is writing a neutral review based of the information given of the product. Information: '{input}'"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [289]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    template="Make a summary of the given review so that the producer can learn what to improve and what already works. Review: '{input}'"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [291]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [295]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
I recently purchased the product with a size of 6.1 inches in Midnight Black for $699. The size was suitable for my needs and the color was sleek and stylish. However, I found the price to be a bit high compared to other similar products on the market. Overall, I am satisfied with the product but would have preferred a lower price point.
Summary: The reviewer found the size and color of the product satisfactory, but felt that the price was too high compared to similar products on the market. They are overall satisfied with the product but suggest lowering the price to make it more competitive.

> Finished chain.


'Summary: The reviewer found the size and color of the product satisfactory, but felt that the price was too high compared to similar products on the market. They are overall satisfied with the product but suggest lowering the price to make it more competitive.'

**Repeat the above twice for different products**

## SequentialChain

In [299]:
from langchain.chains import SequentialChain

In [301]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
    "Translate the following review into Spanish: \n review: {review}\nTranslation"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="spanish_review" #Give a name to your output
                    )


In [303]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "Summarize the following review:\nReview: {review}\nSummary:"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [305]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into English:\n\nReview: {spanish_review}\n\nTranslation:"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="ingles_review"
                      )


In [307]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Based on the Spanish translation, summary, and English translation, craft a follow-up message:\n"
        "Translation: {spanish_review}\n"
        "Summary: {summary}\n"
        "Original Language: {ingles_review}\n\n"
        "Follow-up Message:"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [311]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["spanish_review", "summary", "ingles_review"],
    verbose=True
)

In [313]:
review = df.Review[5]
overall_chain({"review":review})

C:\Users\Ignacio\AppData\Local\Temp\ipykernel_44480\3544409936.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain({"review":review})




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'spanish_review': 'Opinión: Encuentro el sabor mediocre. La espuma no se mantiene, es extraño. Compro los mismos en el mercado y el sabor es mucho mejor... ¿Lote viejo o falsificación!?',
 'summary': 'The reviewer finds the taste mediocre and the foam does not last, which they find strange. They have bought the same product in stores and found the taste to be much better, leading them to suspect they may have received an old batch or a counterfeit product.',
 'ingles_review': "Review: Opinion: I find the taste mediocre. The foam doesn't last, it's strange. I buy the same ones at the market and the taste is much better... Old batch or fake?!"}

**Repeat the above twice for different products or reviews**

## Router Chain

In [375]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are a very knowledgeable biology expert. \
You excel at explaining concepts in biology clearly and concisely. \
When you don't know the answer, you admit it.

Here is a question:
{input}"""

fun_facts_template = """You are an expert in sharing fun and interesting facts. 
When given a topic or a question, you respond with an exciting fact that can 
amaze and entertain the user. Make sure the fact is relevant and surprising.

Here is a topic or question:
{input}
"""

philosophy_template = """You are a philosopher with profound insight into 
life, morality, and the universe. When someone asks you a question, you 
respond with a deep and reflective answer, often drawing on philosophical 
principles or historical philosophers' thoughts.

Here is a question:
{input}
"""

jokes_template = """You are a comedian who specializes in topic-based jokes. 
When given a question or a topic, you respond with a light-hearted and funny 
joke that makes people smile.

Here is the topic:
{input}
"""

quiz_template = """You are a quizmaster who loves testing people’s knowledge. 
When given a topic, you create one or two interesting questions to quiz 
the user. Make the questions that are extremely easy to answer and give said answers.

Here is the topic:
{input}
"""

creative_writing_template = """You are a master storyteller and poet. 
When someone gives you a prompt or idea, you respond with a short story, 
poem, or other creative writing piece that is engaging and imaginative.

Here is the prompt:
{input}
"""

In [377]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering questions about biology",
        "prompt_template": biology_template
    },
    {
        "name": "fun_facts",
        "description": "Good for giving random fun facts",
        "prompt_template": fun_facts_template
    },
    {
        "name": "philosophy",
        "description": "Good for answering philosophy questions",
        "prompt_template": philosophy_template
    },
    {
        "name": "jokes",
        "description": "Good for answering joke questions",
        "prompt_template": jokes_template
    },
    {
        "name": "quiz",
        "description": "Good for giving and answering quiz questions",
        "prompt_template": quiz_template
    },
    {
        "name": "creative_writing",
        "description": "Good for answering and asisting in creative writing questions or promts",
        "prompt_template": creative_writing_template
    },
]

In [379]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [381]:
llm = ChatOpenAI(temperature=0)

In [383]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [385]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [387]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
Do NOT choose any prompt name that is not listed below. \
If the input does not fit any of the given prompts, choose "DEFAULT".

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [389]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [391]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [393]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [395]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [397]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for making proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for passing on genetic information from one generation to the next through the process of cell division and reproduction. This ensures that the genetic information is preserved and maintained in all cells of the body.'

In [399]:
chain.run("what happened in the years 1984 and 1999?")



> Entering new MultiPromptChain chain...
History: {'input': 'what happened in the years 1984 and 1999?'}
> Finished chain.


'In 1984, several significant events took place around the world. Some of the notable events include:\n\n- The assassination of Indian Prime Minister Indira Gandhi by her Sikh bodyguards, leading to widespread anti-Sikh riots in India.\n- The Bhopal gas tragedy in India, where a gas leak from a pesticide plant killed thousands of people and affected many more.\n- The Summer Olympics were held in Los Angeles, USA, boycotted by the Soviet Union and other Eastern Bloc countries in retaliation for the US-led boycott of the 1980 Moscow Olympics.\n- The release of the Apple Macintosh computer, which revolutionized the personal computer industry with its graphical user interface.\n\nIn 1999, some significant events include:\n\n- The Columbine High School massacre in Colorado, USA, where two students killed 13 people and injured many others before committing suicide.\n- The NATO bombing of Yugoslavia during the Kosovo War, leading to the eventual withdrawal of Serbian forces from Kosovo.\n- Th

In [401]:
chain.run("how can I make a webpage")



> Entering new MultiPromptChain chain...
computer science: {'input': 'how can I make a webpage'}
> Finished chain.


'There are several ways to create a webpage, but one common method is to use HTML, CSS, and JavaScript. Here is a step-by-step guide on how to create a simple webpage:\n\n1. Start by creating a new HTML file. You can do this by opening a text editor (such as Notepad or Sublime Text) and saving the file with a .html extension.\n\n2. In the HTML file, start by adding the basic structure of a webpage using the following code:\n\n```html\n<!DOCTYPE html>\n<html>\n<head>\n<title>Your Page Title</title>\n</head>\n<body>\n\n</body>\n</html>\n```\n\n3. Inside the `<body>` tags, you can start adding content to your webpage. This can include text, images, links, and other elements. For example, you can add a heading using the `<h1>` tag:\n\n```html\n<h1>Hello, World!</h1>\n```\n\n4. You can style your webpage using CSS. You can either include CSS directly in your HTML file using the `<style>` tag, or you can create a separate CSS file and link it to your HTML file using the `<link>` tag. Here is

In [403]:
chain.run("tell me a fun fact about eggs")



> Entering new MultiPromptChain chain...
fun_facts: {'input': 'tell me a fun fact about eggs'}
> Finished chain.


'Did you know that the color of an eggshell is determined by the breed of the chicken? Chickens with white feathers and earlobes typically lay white eggs, while chickens with red feathers and earlobes lay brown eggs. However, the color of the eggshell does not affect the taste or nutritional value of the egg inside!'

In [407]:
chain.run("tell me 10 names of popular philosophy schoolars")



> Entering new MultiPromptChain chain...
philosophy: {'input': 'tell me 10 names of popular philosophy scholars'}
> Finished chain.


'1. Plato - A Greek philosopher known for his dialogues and his theory of forms.\n2. Aristotle - A student of Plato and a philosopher who made significant contributions to logic, metaphysics, ethics, and politics.\n3. Immanuel Kant - A German philosopher known for his work on ethics, metaphysics, and epistemology, particularly his concept of the categorical imperative.\n4. Friedrich Nietzsche - A German philosopher known for his critiques of traditional morality and his concept of the "will to power."\n5. Jean-Paul Sartre - A French existentialist philosopher known for his work on freedom, responsibility, and the concept of "bad faith."\n6. Simone de Beauvoir - A French existentialist philosopher known for her work on feminism and existential ethics.\n7. John Stuart Mill - A British philosopher known for his work on utilitarianism and liberalism.\n8. David Hume - A Scottish philosopher known for his empiricist approach to philosophy and his skepticism towards metaphysical concepts.\n9.

In [411]:
chain.run("tell me a joke about a chicken cross the road")



> Entering new MultiPromptChain chain...
jokes: {'input': 'tell me a joke about a chicken cross the road'}
> Finished chain.


'Why did the chicken cross the road? To prove to the possum that it could be done!'

In [413]:
chain.run("give me a quiz about pokemon")



> Entering new MultiPromptChain chain...
quiz: {'input': 'give me a quiz about pokemon'}
> Finished chain.


'Question 1: What is the name of the most famous Pokemon, known for its lightning bolt-shaped tail?\nAnswer: Pikachu\n\nQuestion 2: What type of Pokemon is Charmander?\nAnswer: Fire'

In [415]:
chain.run("how to I help my kid to stop crying?")



> Entering new MultiPromptChain chain...
None: {'input': 'how to I help my kid to stop crying?'}
> Finished chain.


"1. Comfort and reassure them: Offer words of comfort and reassurance to let them know that everything will be okay. Hold them close and provide physical comfort such as hugs or gentle pats on the back.\n\n2. Listen to their feelings: Encourage your child to express their feelings and listen attentively to what they have to say. Let them know that it's okay to feel sad or upset and that you are there to support them.\n\n3. Distract them: Engage your child in a fun or enjoyable activity to help take their mind off of what is upsetting them. Play a game, read a book, or watch a movie together to help distract them from their tears.\n\n4. Offer solutions: If your child is upset about a specific problem or situation, help them brainstorm possible solutions or ways to cope with their emotions. Encourage them to think positively and focus on what they can control.\n\n5. Be patient and understanding: It's important to be patient and understanding with your child as they work through their emo

**Repeat the above at least once for different inputs and chains executions - Be creative!**